In [ ]:
from datetime import date, datetime
import pandas as pd
from pytz import timezone
from alpaca_trade_api.rest import REST, URL, TimeFrame
from liualgotrader.common import config

In [ ]:
nyc = timezone("America/New_York")

In [ ]:
alpaca_rest_client = REST(
    key_id=config.alpaca_api_key, secret_key=config.alpaca_api_secret
)

### Calendar

In [ ]:
since = alpaca_rest_client.get_calendar('2021-11-10', '2021-11-24')
print(since[-1])
print(since[0])
print(len(since))
print(nyc.localize(datetime.combine(since[0].date.date(),since[0].open)))

In [ ]:
print((len(since))*(20-4)*60)

### snapshots

In [ ]:
data = alpaca_rest_client.get_snapshot("AAPL")
print(data.minute_bar)

In [ ]:
data = alpaca_rest_client.get_snapshots(["AAPL"])
print(str(data['AAPL'].latest_trade))

In [ ]:
alpaca_rest_client.list_assets(status='active')

In [ ]:
symbols = [
    asset.symbol
    for asset in alpaca_rest_client.list_assets(
        status="active", asset_class="us_equity"
    )
    if asset.tradable == True
]

In [ ]:
chunk_size = 1000
chunked_symbols = [symbols[i:i+chunk_size] for i in range(0, len(symbols), chunk_size)]
len(chunked_symbols)

In [ ]:
for chunk in chunked_symbols:
    data = alpaca_rest_client.get_snapshots(chunk)
    print(data)

### Crypto


In [ ]:
bars = alpaca_rest_client.get_bars(
    symbol="USDBTC",
    timeframe=TimeFrame.Minute,
    start=date(year=2021, month=4, day=1),
    end=date(year=2021, month=4, day=2),
)
bars

### bars

In [ ]:
bars = alpaca_rest_client.get_bars(
    symbol="AAPL",
    timeframe="1Day",
    start="2020-10-01T00:00:00-04:00",
    end="2020-10-02T00:00:00-04:00",
    limit=10000000,
    adjustment="all").df
bars.index = bars.index.tz_convert('America/New_York') 

In [ ]:
bars

In [ ]:
bars.loc[['2020-12-04 05:42:00-05:00']]

### multi symbol bars

In [ ]:
symbols = ["AAPL", "IBM"]
bars = alpaca_rest_client.get_bars(
    symbol=symbols,
    timeframe=TimeFrame.Hour,
    start=date(year=2019, month=6, day=14),
    end=date(year=2021, month=6, day=14),
)

In [ ]:
bars

In [ ]:
df = pd.read_json(json.dumps(bars._raw))
grouped = df.groupby(df['S'])
for symbol in df['S'].unique():
    temporary_df = grouped.get_group(symbol)
    print(symbol)
    print(temporary_df)